Nama Anggota:
- Givari Akbar
- Iqbal Pandu Santoso
- Muhammad Hanif Zuhair
- Narrendra Setyawan Bahar

# Persiapan

## Install Library

In [1]:
%pip install pyserini

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Library

In [15]:
import nltk
from nltk.corpus import stopwords
import json
import os
from pyserini.search.lucene import LuceneSearcher

## Definisi Dokumen

In [16]:
docs = [
  ("d1",  "The cat chased a small mouse into the garden."),
  ("d2",  "A friendly dog played fetch by the river."),
  ("d3",  "BM25 is a ranking function widely used in search engines."),
  ("d4",  "Boolean retrieval uses logical operators like AND and OR."),
  ("d5",  "TF-IDF weights terms by frequency and rarity."),
  ("d6",  "Neural retrieval uses dense embeddings for semantic search."),
  ("d7",  "The dog and the cat slept on the same couch."),
  ("d8",  "The library hosts a workshop on information retrieval."),
  ("d9",  "Students implemented BM25 and compared it with TF-IDF."),
  ("d10", "The chef roasted chicken with rosemary and garlic."),
  ("d11", "A black cat crossed the old stone bridge at night."),
  ("d12", "Dogs are loyal companions during long hikes."),
  ("d13", "The dataset contains fifteen short sentences for testing."),
  ("d14", "Reranking models reorder BM25 candidates using transformers."),
  ("d15", "The dog sniffed a cat but ignored the mouse.")
]

# Preprocessing

In [21]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to C:\Users\MyBook Hype
[nltk_data]     AMD\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\MyBook Hype
[nltk_data]     AMD\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\MyBook Hype
[nltk_data]     AMD\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [18]:
def preprocessing(docs):
    preprocessed_docs = []
    stop_words = set(stopwords.words('english'))
    for doc_id, doc in docs:
        # Mengubah ke lowecase
        doc = doc.lower()
        tokens = nltk.word_tokenize(doc)
        # Menghilangkan stopwords
        filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

        cleaned_doc = ' '.join(filtered_tokens)
        preprocessed_docs.append((doc_id, cleaned_doc))
    return preprocessed_docs

In [25]:
docs_res = preprocessing(docs)
for docs_id, doc in docs_res:
    print(f"{docs_id}: {doc}")

d1: cat chased small mouse garden
d2: friendly dog played fetch river
d3: ranking function widely used search engines
d4: boolean retrieval uses logical operators like
d5: weights terms frequency rarity
d6: neural retrieval uses dense embeddings semantic search
d7: dog cat slept couch
d8: library hosts workshop information retrieval
d9: students implemented compared
d10: chef roasted chicken rosemary garlic
d11: black cat crossed old stone bridge night
d12: dogs loyal companions long hikes
d13: dataset contains fifteen short sentences testing
d14: reranking models reorder candidates using transformers
d15: dog sniffed cat ignored mouse


## Indexing Corpus + Stemming

## Menyimpan Hasil Preprocessing ke JSONL 

In [26]:
jsonl_path = 'documents.jsonl'
documents = [{'id': doc_id, 'contents': doc} for doc_id, doc in docs_res]

if not os.path.exists('docs'):
    os.makedirs('docs')

with open(os.path.join('docs', jsonl_path), 'w') as f:
    for doc in documents:
        f.write(json.dumps(doc) + '\n')

## Menjalankan Indexing + Stemming

In [27]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input docs \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --stemmer porter \
  --storePositions --storeDocvectors --storeRaw

2025-08-26 12:17:59,630 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:205) - Setting log level to INFO
2025-08-26 12:17:59,634 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - ============ Loading Index Configuration ============
2025-08-26 12:17:59,635 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) - AbstractIndexer settings:
2025-08-26 12:17:59,635 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + DocumentCollection path: docs
2025-08-26 12:17:59,635 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + CollectionClass: JsonCollection
2025-08-26 12:17:59,636 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Index path: indexes/sample_collection_jsonl
2025-08-26 12:17:59,637 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Threads: 1
2025-08-26 12:17:59,638 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:214) -  + Optimize (merge segments)? false
2025-08-26 12:17:59,704 I

Aug 26, 2025 12:17:59 PM org.apache.lucene.store.MMapDirectory lookupProvider
